In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tensorflow.keras import layers, Model
import seaborn as sns
import glob

In [2]:
# 1. Find all .xls files in the folder
folder = "./23RAMA"
paths = glob.glob(os.path.join(folder, "*.xls"))

# 2. Load each file into a DataFrame, store in a dict for easy reference
dfs = {os.path.basename(p): pd.read_excel(p) for p in paths}

# 3. Compute the intersection of columns across all DataFrames
common_cols = set(dfs[next(iter(dfs))].columns)  # start with the first file’s columns
for df in dfs.values():
    common_cols &= set(df.columns)
common_cols = sorted(common_cols)  # optional: sort alphabetically

# 4. Filter each DataFrame to keep only the common columns
for name, df in dfs.items():
    dfs[name] = df.loc[:, common_cols]

# Now `dfs` holds your cleaned DataFrames, keyed by filename.
# Example: to access the cleaned DataFrame for 'file1.xls':
cleaned_df = dfs["2023CO.xls"]
cleaned_df.head()  # Display the first few rows of the cleaned DataFrame


,AJM,BJU,CAM,FAR,FECHA,HGM,HORA,INN,MER,MGH,MPA,PED,SAC,SAG,SFE,TLA,UIZ,XAL
0,0.45,1.24,1.51,1.00,2023-01-01,0.80,1,0.20,1.71,1.2,0.15,0.46,0.45,1.70,-99,0.90,1.02,-99.0
1,0.43,1.53,1.66,1.49,2023-01-01,1.11,2,0.16,1.39,1.0,0.18,0.34,0.68,1.45,-99,0.74,0.87,-99.0
2,0.42,1.41,1.56,1.31,2023-01-01,1.49,3,0.13,1.14,1.2,0.17,0.40,1.94,1.46,-99,0.75,0.99,-99.0
3,0.48,1.14,1.43,1.11,2023-01-01,1.59,4,0.12,1.25,1.1,0.19,0.38,2.46,1.64,-99,0.89,1.20,-99.0
4,0.37,0.98,1.52,1.24,2023-01-01,1.06,5,0.11,1.38,1.3,0.16,0.47,2.96,1.69,-99,1.11,1.12,-99.0


In [3]:
import pandas as pd
from functools import reduce

long_tables = []

for var_name, df in dfs.items():
    # 1. Parse FECHA to datetime (dayfirst if your dates are dd/mm/YYYY)
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    
    # 2. Convert HORA (int or str) into a timedelta of hours
    df['HORA'] = pd.to_numeric(df['HORA'], errors='coerce').fillna(0).astype(int)
    df['timestamp'] = df['FECHA'] + pd.to_timedelta(df['HORA'], unit='h')
    
    # 3. Melt each station column into rows
    station_cols = [c for c in df.columns if c not in ('FECHA','HORA','timestamp')]
    df_long = df.melt(
        id_vars='timestamp',
        value_vars=station_cols,
        var_name='station',
        value_name=var_name
    )
    
    # 4. Keep just timestamp, station, and the variable
    long_tables.append(df_long[['timestamp','station', var_name]])

# 5. Inner-merge on (timestamp, station) across all variables
big_df = reduce(
    lambda left, right: pd.merge(left, right, on=['timestamp','station'], how='inner'),
    long_tables
)

# 6. Extract feature matrix for your autoencoder
X = big_df.drop(columns=['timestamp','station']).values


In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
import tensorflow as tf
n_features = X_scaled.shape[1]

# Encoder
inp = layers.Input(shape=(n_features,))
h = layers.Dense(64, activation='relu')(inp)
h = layers.Dense(32, activation='relu')(h)
encoded = layers.Dense(2, activation=None, name='bottleneck')(h)

# Decoder
h = layers.Dense(32, activation='relu')(encoded)
h = layers.Dense(64, activation='relu')(h)
decoded = layers.Dense(n_features, activation=None)(h)

# Full autoencoder
autoencoder = Model(inp, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train
history = autoencoder.fit(
    X_scaled, X_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

# Extract the encoder model
encoder = Model(inp, encoded)


2025-04-29 22:42:36.080385: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-29 22:42:36.080421: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-29 22:42:36.080424: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-29 22:42:36.080441: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-29 22:42:36.080452: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2025-04-29 22:42:36.621407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


2290/2290 ━━━━━━━━━━━━━━━━━━━━ 35s 15ms/step - loss: 34.2521 - val_loss: 8.4340
Epoch 2/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - loss: 305.5733 - val_loss: 935.5275
Epoch 3/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 33s 14ms/step - loss: 711.7605 - val_loss: 1254.3257
Epoch 4/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 33s 14ms/step - loss: 803.7786 - val_loss: 83.7353
Epoch 5/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 34s 15ms/step - loss: 1540.5088 - val_loss: 463.2620
Epoch 6/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 34s 15ms/step - loss: 1107.9438 - val_loss: 6.1162
Epoch 7/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 34s 15ms/step - loss: 3049.0862 - val_loss: 6290.9399
Epoch 8/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 34s 15ms/step - loss: 4212.8657 - val_loss: 4585.2329
Epoch 9/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - loss: 4403.9790 - val_loss: 8.7838
Epoch 10/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 33s 14ms/step - loss: 2664.7202 - val_loss: 589.5716
Epoch 11/100
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 33s 14ms/step - 

In [6]:
import matplotlib.pyplot as plt

# 4a. Get 2D codes for each sample
codes2d = encoder.predict(X_scaled)  # shape (n_samples, 2)

# 4b. Recover station and timestamp labels
labels = big_df.index.to_frame(index=False)  # DataFrame with columns ['timestamp', 'station']

# 4c. Scatter-plot, color by station (or other grouping)
plt.figure(figsize=(8, 6))
for station, group in pd.DataFrame(codes2d, index=labels['station']).groupby(level=0):
    pts = group.values
    plt.scatter(pts[:, 0], pts[:, 1], label=station, s=10)

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('Autoencoder embedding of climate data')
plt.legend(bbox_to_anchor=(1.05, 1), fontsize='small')
plt.tight_layout()
plt.show()


2544/2544 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


KeyError: 'station'

<Figure size 800x600 with 0 Axes>